# Data Cleaning and Preperation
The purpose of this notebook is to showcase the process that was undertaken to clean and prepare the data to be fed into a linear regression model. Further preperation may be required when building the categorical models in future parts of the project.

I began the process by importing the required libraries and loading the data.

In [1]:
#Import libraries 
import pandas as pd
import re

In [2]:
#Read data
y1 = pd.read_csv('Data/2015.csv')
y2 = pd.read_csv('Data/2016.csv')
y3 = pd.read_csv('Data/2017.csv')

Since many processes will have to be repeated for all three files I will defined some usefull functions to avoiding having to repeat myself. 

In [3]:
def add_column(df, col, val):
    df[col] = val
    
def remove_puncuation(df):
    '''Removes puncuation from column names based off regular expressions'''
    new_cols = []
    for col in df.columns:
        col = re.sub('[.!#?,:;]', '', col)
        new_cols.append(col)
        
    df.columns = new_cols
    
def clean_columns(df, ops):
    '''Performs user defined operations across all columns'''
    new_columns = []
    columns = list(df.columns)
    for col in columns:
        for function in ops:
            col = function(col)
        new_columns.append(col)
        
    df.columns = new_columns 
  
#operations to be performed on columns
clean_ops = [str.strip, str.title]

I then checked for missing values and applied these functions on all three files.

 ## 2015

In [4]:
#Check for missing values
y1.isnull().values.any()

False

In [5]:
y1.isna().values.any()

False

In [6]:
#Add year column
add_column(y1, 'Year', 2015)
remove_puncuation(y1)
clean_columns(y1, clean_ops)

## 2016

In [7]:
#Check for missing values
y2.isnull().values.any()

False

In [8]:
y2.isna().values.any()

False

In [9]:
#add year columnn
add_column(y2, 'Year', 2016)
remove_puncuation(y2)
clean_columns(y2, clean_ops)

## 2017

In [10]:
#check for null values
y3.isnull().values.any() 

False

In [11]:
y3.isna().values.any()

False

In [12]:
#add year column
add_column(y3, 'Year', 2017)
remove_puncuation(y3)
clean_columns(y3, clean_ops)

Luckily, none of the files contained any missing values. Making the data cleaning process much easier.

# Creating One Giant Data Frame 
Once the files were cleaned at a surface level, I began the process of manipulating them into structures that could be concatenated into one giant dataframe.

To start, I created a datframe containing the column names into one giant dataframe.

In [13]:
y1_columns = pd.Series(y1.columns)
y2_columns = pd.Series(y2.columns)
y3_columns = pd.Series(y3.columns)

all_columns = pd.DataFrame({'2015': (y1_columns), '2016': y2_columns, '2017': y3_columns})
all_columns = all_columns.sort_values('2015')

Then I redefined the column names to meet a standardized format.

In [14]:
y1.rename(columns = {'Year': 'Year',
                     'Country': 'Country', 
                     'Economy (Gdp Per Capita)': 'GDP Per Capita', 
                     'Happiness Rank': 'Happiness Rank', 
                     'Happiness Score' : 'Happiness Score', 
                     'Health (Life Expectancy)': 'Life Expectancy', 
                     'Trust (Government Corruption)': 'Government Corruption', 
                     'Family': 'Family', 
                     'Freedom': 'Freedom', 
                     'Generosity': 'Generosity'
                    }, inplace = True)

y2.rename(columns = {'Year': 'Year', 
                     'Country': 'Country', 
                     'Economy (Gdp Per Capita)': 'GDP Per Capita', 
                     'Happiness Rank': 'Happiness Rank',
                     'Happiness Score': 'Happiness Score', 
                     'Family': 'Family', 
                     'Health (Life Expectancy)': 'Life Expectancy', 
                     'Trust (Government Corruption)': 'Government Corruption', 
                     'Freedom': 'Freedom', 
                     'Generosity': 'Generosity' 
                    }, inplace = True)

y3.rename(columns = {'Year': 'Year', 
                     'Country': 'Country', 
                     'Economygdppercapita' : 'GDP Per Capita', 
                     'Happinessrank': 'Happiness Rank', 
                     'Happinessscore': 'Happiness Score', 
                     'Family': 'Family', 
                     'Healthlifeexpectancy': 'Life Expectancy', 
                     'Trustgovernmentcorruption': 'Government Corruption', 
                     'Freedom': 'Freedom', 
                     'Generosity': 'Generosity'
                    }, inplace = True)

Once the column names were all in a standardized format, I defined which column names to keep then created a function that drops all the columns that I did not make the cut and reorders the columns.

Columns were selected based off similarity across the 3 datasets not preference or bias.

In [15]:
true_cols = ['Year', 'Country', 'GDP Per Capita', 'Happiness Rank', 
             'Happiness Score', 'Life Expectancy', 'Government Corruption', 
             'Family', 'Freedom', 'Generosity']

def drop_columns(df, true_cols):
    """Drops columns we don't want from each data frame"""
    drop_columns = []
    for df_col in list(df.columns):
        for true_col in true_cols:
            if df_col not in true_cols:
                drop_columns.append(df_col)
    df.drop(columns = drop_columns, inplace = True)

#Drop columns from each dataframe
drop_columns(y1, true_cols)
drop_columns(y2, true_cols)
drop_columns(y3, true_cols)

Finally, once all datframes were in the correct format. I concatenated them into one giant dataframe, reordered it based off the year the data was collected, then wrote it into a csv for future use.